In [ ]:
# default_exp dash

# The Movie Database client

> A simple client to work with the TMDB API

https://themoviedb.org

In [ ]:
#hide
from nbdev.export import *

In [ ]:
#export
from dataclasses import dataclass
from functools import lru_cache

import requests

import pandas as pd

In [ ]:
# export
def load_api_key():
    import os
    key = os.environ.get('TMDB_API_KEY')
    if key:
        return key
    raise RuntimeError("TMDB_API_KEY is not defined")

In [ ]:
@dataclass
class MovieData:
    movie_id: int
    tmdb_id: int
    title: str
    image_url: str

In [ ]:
#export
class TmdbApi:
    def __init__(self, api_key: str, movielens_to_tmdb_lookup: dict):
        self.api_key = api_key
        self.movielens_to_tmdb_lookup = movielens_to_tmdb_lookup
        self.tmdb_to_movielens_lookup = {tmdb_id:movielens_id for movielens_id, tmdb_id in movielens_to_tmdb_lookup.items() }

    @lru_cache(maxsize=100_000)
    def fetch_movie_data_by_tmdb_id(self, tmdb_movie_id: int) -> MovieData:
        data = requests.get(f"https://api.themoviedb.org/3/movie/{tmdb_movie_id}?api_key={self.api_key}").json()
        poster_path = data.get("poster_path")
        return MovieData(
            movie_id=self.tmdb_to_movielens_lookup.get(tmdb_movie_id),
            tmdb_id=self.tmdb_to_movielens_lookup.get(tmdb_movie_id),
            title=data.get('title'),
            image_url=f'https://image.tmdb.org/t/p/w185{poster_path}'
        )

    def fetch_movie_data_by_movielens_id(self, movielens_id: int) -> MovieData:
        tmdb_id = self.movielens_to_tmdb_lookup.get(movielens_id)
        return self.fetch_movie_data_by_tmdb_id(movielens_id)


In [ ]:
links_df = pd.read_csv("./data/ml-latest-small/links.csv")
links_df

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [ ]:
movielens_to_tmdb_lookup = { int(movie_id):int(tmdb_id)  for movie_id, tmdb_id in links_df.set_index("movieId")["tmdbId"].to_dict().items() if movie_id > 0 and tmdb_id > 0 }
list(movielens_to_tmdb_lookup.items())[:10]

[(1, 862),
 (2, 8844),
 (3, 15602),
 (4, 31357),
 (5, 11862),
 (6, 949),
 (7, 11860),
 (8, 45325),
 (9, 9091),
 (10, 710)]

In [ ]:
api = TmdbApi(api_key=load_api_key(), movielens_to_tmdb_lookup=movielens_to_tmdb_lookup)

In [ ]:
tmdb_movie_id=550
requests.get(f"https://api.themoviedb.org/3/movie/{tmdb_movie_id}?api_key={load_api_key()}").json()

{'adult': False,
 'backdrop_path': '/mMZRKb3NVo5ZeSPEIaNW9buLWQ0.jpg',
 'belongs_to_collection': None,
 'budget': 63000000,
 'genres': [{'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.foxmovies.com/movies/fight-club',
 'id': 550,
 'imdb_id': 'tt0137523',
 'original_language': 'en',
 'original_title': 'Fight Club',
 'overview': 'A ticking-time-bomb insomniac and a slippery soap salesman channel primal male aggression into a shocking new form of therapy. Their concept catches on, with underground "fight clubs" forming in every town, until an eccentric gets in the way and ignites an out-of-control spiral toward oblivion.',
 'popularity': 38.803,
 'poster_path': '/adw6Lq9FiC9zjYEpOqfq03ituwp.jpg',
 'production_companies': [{'id': 508,
   'logo_path': '/7PzJdsLGlR7oW4J0J5Xcd0pHGRg.png',
   'name': 'Regency Enterprises',
   'origin_country': 'US'},
  {'id': 711,
   'logo_path': '/tEiIH5QesdheJmDAqQwvtN60727.png',
   'name': 'Fox 2000 Pictures',
   'origin_country': 'US'},
  {'id': 205

In [ ]:
api.fetch_movie_data_by_tmdb_id(tmdb_movie_id)

MovieData(movie_id=2959, tmdb_id=2959, title='Fight Club', image_url='https://image.tmdb.org/t/p/w185/adw6Lq9FiC9zjYEpOqfq03ituwp.jpg')